<h1>Reading Table Data About Postcode & Neighbourhood from Wikipedia web page.</h1>

Created By: Prateek Vashishtha<br>
Last Updated: 31 may 2019<br>
Tools: Jupyter Notebook

Installing BeautifulSoup Module to parse web page of Wikipedia

In [ ]:
!conda install --yes BeautifulSoup4

In [1]:
from bs4 import BeautifulSoup #importing BeautifulSoup ver-4

In [2]:
import requests

In [3]:
web_data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup=BeautifulSoup(web_data,'lxml')

Fetchinf table

In [5]:
table = soup.find('table',{'class':'wikitable sortable'})

Splitting into rows

In [8]:
rows=soup.findAll('tr')

Finding Table Headings to be used as column names in datafraem later

In [9]:
cols=soup.findAll('th')

In [10]:
df_columns=[cols[0].text.strip(),cols[1].text.strip(),cols[2].text.strip()]

Fetching Cell Data from Table Data

In [11]:
cells=soup.findAll('td')

In [12]:
cell_data=[]
for data in cells:
    cell_data.append(data.text.strip())

In [13]:
cell_data=cell_data[:864]

Making Data Frame Structure out of list

In [14]:
import pandas as pd
list1=[]
list2=[]
list3=[]
for i in range(0,864,3):
    list1.append(cell_data[i])
    list2.append(cell_data[i+1])
    list3.append(cell_data[i+2])
    
df=pd.DataFrame(list(zip(list1,list2,list3)))
df.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


making data frame as required in rpoblem

In [15]:
df.columns=df_columns

removing rows having 'Not assigned' as values for Borough column

In [16]:
df=df[df['Borough']!='Not assigned']

Grouping to postcode and merginf neighbourhood data into single row seperated by comma

In [17]:
temp=pd.DataFrame()
temp=df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join)

In [18]:
temp=pd.DataFrame(temp)

In [19]:
temp=temp.reset_index(level=['Postcode','Borough'])

In [20]:
temp.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

Final data cleaning and creaating final dataframe to store data processed

In [21]:
for i in range(temp.shape[0]):
    if temp['Neighbourhood'][i]=='Not assigned':
        temp['Neighbourhood'][i]=temp['Borough'][i]

In [22]:
final_df=pd.DataFrame()
final_df=temp.copy()

Checking data frame shape

In [23]:
final_df.shape

(103, 3)